In [1]:
import requests

def get_wind_forecast(lat, lon):
    """
    Fetch wind forecast data from NOAA's NWS API.

    Parameters:
        lat (float): Latitude of the location.
        lon (float): Longitude of the location.

    Returns:
        dict: Wind forecast data.
    """
    # Base URL for NWS Points Forecast API
    base_url = "https://api.weather.gov/points"
    
    try:
        # Step 1: Get the forecast URL for the given coordinates
        response = requests.get(f"{base_url}/{lat},{lon}")
        response.raise_for_status()  # Raise exception for HTTP errors
        data = response.json()

        forecast_url = data["properties"]["forecastHourly"]
        
        # Step 2: Fetch hourly forecast data
        forecast_response = requests.get(forecast_url)
        forecast_response.raise_for_status()
        forecast_data = forecast_response.json()

        # Step 3: Extract wind forecast information
        wind_forecast = []
        for period in forecast_data["properties"]["periods"]:
            wind_forecast.append({
                "time": period["startTime"],
                "wind_speed": period["windSpeed"],
                "wind_direction": period["windDirection"]
            })

        return wind_forecast
    
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None



In [2]:

# Replace with your desired coordinates
latitude = 33.971727  # Example: Los Angeles
longitude = -118.4556527

wind_data = get_wind_forecast(latitude, longitude)
if wind_data:
    print("Wind Forecast:")
    for entry in wind_data:
        print(f"Time: {entry['time']}, Speed: {entry['wind_speed']}, Direction: {entry['wind_direction']}")


Wind Forecast:
Time: 2024-12-03T10:00:00-08:00, Speed: 5 mph, Direction: WSW
Time: 2024-12-03T11:00:00-08:00, Speed: 5 mph, Direction: WSW
Time: 2024-12-03T12:00:00-08:00, Speed: 5 mph, Direction: WSW
Time: 2024-12-03T13:00:00-08:00, Speed: 5 mph, Direction: WSW
Time: 2024-12-03T14:00:00-08:00, Speed: 5 mph, Direction: WSW
Time: 2024-12-03T15:00:00-08:00, Speed: 5 mph, Direction: WSW
Time: 2024-12-03T16:00:00-08:00, Speed: 10 mph, Direction: W
Time: 2024-12-03T17:00:00-08:00, Speed: 10 mph, Direction: W
Time: 2024-12-03T18:00:00-08:00, Speed: 10 mph, Direction: W
Time: 2024-12-03T19:00:00-08:00, Speed: 10 mph, Direction: W
Time: 2024-12-03T20:00:00-08:00, Speed: 10 mph, Direction: W
Time: 2024-12-03T21:00:00-08:00, Speed: 10 mph, Direction: W
Time: 2024-12-03T22:00:00-08:00, Speed: 0 mph, Direction: 
Time: 2024-12-03T23:00:00-08:00, Speed: 0 mph, Direction: 
Time: 2024-12-04T00:00:00-08:00, Speed: 0 mph, Direction: 
Time: 2024-12-04T01:00:00-08:00, Speed: 0 mph, Direction: 
Time: 2024-

In [15]:
import requests
import datetime

def download_grib_file(min_lat, max_lat, min_lon, max_lon, forecast_hour, output_file):
    """
    Downloads a GRIB2 file from NOAA for a specified area and forecast hour.

    Parameters:
        min_lat (float): Minimum latitude of the area.
        max_lat (float): Maximum latitude of the area.
        min_lon (float): Minimum longitude of the area.
        max_lon (float): Maximum longitude of the area.
        forecast_hour (int): Forecast hour (e.g., 240 for 10-day forecast).
        output_file (str): Path to save the downloaded GRIB2 file.
    """
    # Get the current UTC time
    now = datetime.datetime.now(datetime.timezone.utc)
    run_date = now.strftime("%Y%m%d")
    run_hour = "00"  # Default to 00 UTC; change as needed

    # Construct the URL for the GRIB2 file
    file_name = f"gfs.t{run_hour}z.pgrb2b.0p25.f{forecast_hour:03d}"
    base_url = f"https://nomads.ncep.noaa.gov/pub/data/nccf/com/gfs/prod/"
    file_url = f"{base_url}gfs.{run_date}/{run_hour}/atmos/{file_name}"

    try:
        # Request to download the GRIB2 file
        response = requests.get(file_url, stream=True)
        response.raise_for_status()  # Check for HTTP errors

        # Save the file locally
        with open(output_file, "wb") as file:
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)

        print(f"GRIB2 file downloaded successfully: {output_file}")
    except requests.exceptions.RequestException as e:
        print(f"Error downloading GRIB2 file: {e}")
        print(f"URL attempted: {file_url}")


In [16]:

# Example usage
# Define the geographic bounds (Note: bounds are ignored for GRIB2b files)
min_lat = 32.0
max_lat = 34.5
min_lon = -122.0
max_lon = -116.0

# Specify the forecast hour (e.g., 240 for 10 days) and output file
forecast_hour = 240
output_path = "gfs_10day_forecast.grib2"

# Download the GRIB2 file
download_grib_file(min_lat, max_lat, min_lon, max_lon, forecast_hour, output_path)


GRIB2 file downloaded successfully: gfs_10day_forecast.grib2
